In [1]:
import numpy
import numpy as np
import math
import pandas as pd
from scipy.optimize import minimize
from gt_design import *

## Define Ranges

Define the range for the stage loading coefficient, flow coefficient and the reaction. Using the obtained values, calculate the other values.

Use the following data:
1. Reaction ($\Lambda$) -> 0.35 - 0.65
2. Flow Coefficient ($\phi$) -> 0.6 - 1.0
3. Structural Limit ($AN^2$) -> 10000000 - 22580600 (checking for a wide range)
4. Zweifel for vane -> 0.75 - 0.90
5. Zweifel for rotor -> 0.80 - 0.95

In [2]:
range_degree_reaction = numpy.linspace(0.4, 0.6, 8)
range_stage_loading = numpy.linspace(2.5, 5, 10)
range_alpha_3 = numpy.linspace(10, 35, 10)
range_mach_exit = numpy.linspace(0.3, 0.55, 15)
range_AN_squared = numpy.linspace(10000000, 22580600, 10)
range_zweifel_vane = numpy.linspace(0.75, 0.90, 15)
range_zweifel_rotor = numpy.linspace(0.80, 0.95, 15)  

## int main(){}

In [3]:
def int_main(range_degree_reaction, range_stage_loading, range_alpha_3, range_AN_squared, range_zweifel_vane, range_zweifel_rotor):

    data = []
    data_2 = []

    for i in range_mach_exit:
        for j in range_stage_loading:
            for k in range_alpha_3:
                for l in range_degree_reaction:
                    for m in range_AN_squared:
                        #for n in range_zweifel_vane:
                        # MEANLINE ANALYSIS
                        T_1, P_1, rho_1, C_1 = aeroturbine.calc_properties(M_1, T_01, P_01)
                        C_a_1 = C_1 * np.cos(np.radians(-10))
                        T_3, P_3, rho_3, C_3 = aeroturbine.calc_properties(i, T_03, P_03)

                        # Calculate U from the stage loading.
                        U = aeroturbine.calc_U(j)
                        C_a_3, C_w_3,  V_3, V_w_3, flow_coefficient_3, beta_3, a_3, M_3_rel, A_3,P_03_rel = aeroturbine.calc_stage_3(U, C_3, T_3, rho_3,l,P_3,k)
                        T_02,T_2, P_2, rho_2, A_2, C_a_2, flow_coefficient_2, a_2, V_w_2, beta_2, V_2, C_w_2, C_2, alpha_2, M_2, M_2_rel,P_02, P_02_rel = aeroturbine.calc_stage_2(U, l, T_1, T_3, P_3, A_3, V_w_3)
                        
                        # STRUCTURAL ANALYSIS
                        A_1 = m_dot_1/(rho_1 * C_a_1)
                        N, omega, r_hub, r_tip, r_meanline, h = aerostructural.calc_structural(m, A_2, U)

                        if V_3 > V_2 and C_2 > C_1 and rho_3 < rho_2 < rho_1: # Velocity check, density check and omega check _>
                            # Other value checks from the design requirements _>
                            # BLADE VORTEX ANALYSIS
                            alpha_2_hub, alpha_3_hub, beta_2_hub, beta_3_hub, U_hub, V_2_hub, C_2_hub, M_2_rel_hub, M_2_hub, reaction_hub = aeroturbine.calc_hub_angles(r_meanline, r_hub, alpha_2, k, flow_coefficient_2, flow_coefficient_3, U, C_a_2, a_2,T_02,T_03,T_1,C_a_3)
                            alpha_2_tip, alpha_3_tip, beta_2_tip, beta_3_tip, U_tip, V_2_tip, C_2_tip, M_2_rel_tip, M_2_tip = aeroturbine.calc_tip_angles(r_meanline, r_tip, alpha_2, k, flow_coefficient_2, flow_coefficient_3, U, C_a_2, a_2)

                            if 40 < alpha_2 < 70 and U_hub < 335.28  and reaction_hub > 0 and M_2_rel_hub > M_2_rel_tip and 3076 < omega < 4754 and P_03_rel < P_02_rel and P_02 < P_01:
                                # ANGLES CHECK
                                if 0 < alpha_2_tip < alpha_2 < alpha_2_hub and 0 < beta_2_tip < beta_2 < beta_2_hub and 0 < alpha_3_tip < k < alpha_3_hub and beta_3_tip > beta_3 > beta_3_hub > 0 and M_2_hub < 0.95: 

                                    # OFF DESIGN CALCULATIONS
                                    T_3_od,rho_3_od, P_3_od, alpha_3_od, flow_coeff_2_od, incidence_2, v_2_od, alpha_3_od,C_w_3_od,C_a_3_od,U_mean_od,flow_coeff_3_od, work_od_cw, work_od_vw, M_2_rel_od, M_3_rel_od = off_design.calc_off_design(A_3,U, beta_3,C_a_2,C_w_2,beta_2, a_2, a_3)

                                    if flow_coeff_3_od > flow_coeff_2_od and C_a_3_od > 0 and 10 <= alpha_3_od <= 40:

                                        # VANE AREA AND HEIGHT
                                        A_vane_mean = (A_1 + A_2)/2
                                        h_vane_mean = (A_vane_mean * N/60)/U
                                        r_tip_stator = r_meanline + (h_vane_mean/2)
                                        r_hub_stator = r_meanline - (h_vane_mean/2)

                                        # DESIGN POINT LOSSES
                                        # CALCULATE LOSSES - Stator
                                        K_p_stator, pitch_chord_ratio_stator, K_accel_stator, stagger_angle_stator, pitch_chord_ratio_stator, pitch_axial_chord_ratio_stator, K_1_s, K_2_s =  aerodynamic_losses.profile_losses.calc_K_p(M_1, M_2, M_2_hub, P_1, P_2, r_tip_stator, r_hub_stator, -10, alpha_2, 0.75)
                                        K_s_stator = aerodynamic_losses.secondary_losses.calc_K_s(K_accel_stator, AR_vane, -10, alpha_2)
                                        K_TET_stator, N_stator, c_true_stator, c_a_stator = aerodynamic_losses.trailing_edge_losses_stator.K_TET(M_1, -10, alpha_2, h_vane_mean, stagger_angle_stator, r_meanline, pitch_axial_chord_ratio_stator)
                                        K_stator = K_p_stator + K_s_stator + K_TET_stator

                                        # CALCULATE LOSSES - Rotor
                                        K_p_rotor, pitch_chord_ratio_rotor, K_accel_rotor, stagger_angle_rotor, pitch_chord_ratio_rotor, pitch_axial_chord_ratio_rotor, K_1_rotor, K_2_rotor =  aerodynamic_losses.profile_losses.calc_K_p(M_2_rel, M_3_rel, M_2_rel_hub, P_2, P_3, r_tip, r_hub,  beta_2, beta_3, 0.80)
                                        K_s_rotor = aerodynamic_losses.secondary_losses.calc_K_s(K_accel_rotor, AR_rotor, beta_2, beta_3)
                                        K_TET_rotor, N_rotor, c_true_rotor, c_a_rotor = aerodynamic_losses.trailing_edge_losses_rotor.K_TET(M_2_rel, beta_2, beta_3, h, stagger_angle_rotor, r_meanline, pitch_axial_chord_ratio_rotor)
                                        K_rotor = K_p_rotor + K_s_rotor + K_TET_rotor


                                        # EFFICIENCY CALCULATIONS DESIGN POINT
                                        eta_tt = aerodynamic_losses.efficiency_calculations(K_stator, K_rotor, M_2, M_3_rel, C_2, V_3)
                                        delta_n, eta_final = aerodynamic_losses.efficiency_final(eta_tt, h, beta_3, r_tip, r_meanline)

                                        # OFF DESIGN LOSSES
                                        eta_tt_od, delta_n_od, eta_final_od = aerodynamic_losses.losses_off_design(K_p_rotor, K_s_rotor, K_stator, K_1_rotor, K_2_rotor, pitch_chord_ratio_rotor, pitch_axial_chord_ratio_rotor, c_true_rotor, alpha_2, beta_2, beta_3, incidence_2,  M_2_rel_od, M_3_rel_od, h, stagger_angle_rotor, r_meanline, r_tip, C_2, V_3)

                                        # EFFICIENCY CALCULATIONS OFF DESIGN

                                        # WORK CHECKS
                                        work_Part_A = c_p_gas*1000*(T_01-T_03)
                                        work_check_cw = U * (C_w_2 + C_w_3)
                                        work_check_vw = U * (V_w_2 + V_w_3)

                                        if eta_final_od > 0: # This is to filter the abnormal behavior with the graph                                    
                                            data.append((j, l, work_Part_A, work_check_cw, work_check_vw, T_1, P_1, rho_1, A_1, C_1, C_a_1, i, T_3, P_3, rho_3, C_3, k, U,
                                                        C_a_3, C_w_3,  V_3, V_w_3, flow_coefficient_3, beta_3, a_3, M_3_rel, A_3,
                                                        T_2, P_2, P_02, P_02_rel, P_03_rel ,rho_2, A_2, C_a_2, flow_coefficient_2, a_2, V_w_2, beta_2, V_2, C_w_2, C_2, alpha_2, M_2, M_2_rel,
                                                        N, omega, r_hub, r_tip, r_meanline, h,
                                                        alpha_2_hub, alpha_3_hub, beta_2_hub, beta_3_hub, U_hub, V_2_hub, C_2_hub, M_2_rel_hub, M_2_hub, reaction_hub,
                                                        alpha_2_tip, alpha_3_tip, beta_2_tip, beta_3_tip, U_tip, V_2_tip, C_2_tip, M_2_rel_tip, M_2_tip,
                                                        K_p_stator, K_s_stator, K_TET_stator, K_stator, N_stator,
                                                        K_p_rotor, K_s_rotor, K_TET_rotor, K_rotor, N_rotor, K_1_rotor, K_2_rotor, pitch_chord_ratio_rotor, c_true_rotor, c_a_rotor,stagger_angle_rotor,pitch_axial_chord_ratio_rotor,
                                                        eta_tt, delta_n, eta_final,
                                                        eta_tt_od, delta_n_od, eta_final_od,
                                                        work_od_cw, rho_3_od, T_3_od, P_3_od, alpha_3_od, flow_coeff_2_od, incidence_2, v_2_od, beta_3, C_w_3_od, C_a_3_od, U_mean_od, flow_coeff_3_od, M_2_rel_od, M_3_rel_od
                                                        ))
                                            # ===============================================   
                                            # START OF LOOP 2
                                            # Change the incidence angle
                                            beta_2 = beta_2 + 6
                                            T_3_od,rho_3_od, P_3_od, alpha_3_od, flow_coeff_2_od, incidence_2, v_2_od, alpha_3_od,C_w_3_od,C_a_3_od,U_mean_od,flow_coeff_3_od, work_od_cw, work_od_vw, M_2_rel_od, M_3_rel_od = off_design.calc_off_design(A_3,U, beta_3,C_a_2,C_w_2,beta_2, a_2, a_3)
                                            # EFFICIENCY CALCULATIONS DESIGN POINT


                                            # CALCULATE LOSSES - Stator
                                            K_p_stator, pitch_chord_ratio_stator, K_accel_stator, stagger_angle_stator, pitch_chord_ratio_stator, pitch_axial_chord_ratio_stator, K_1_s, K_2_s =  aerodynamic_losses.profile_losses.calc_K_p(M_1, M_2, M_2_hub, P_1, P_2, r_tip_stator, r_hub_stator, -10, alpha_2, 0.75)
                                            K_s_stator = aerodynamic_losses.secondary_losses.calc_K_s(K_accel_stator, AR_vane, -10, alpha_2)
                                            K_TET_stator, N_stator, c_true_stator, c_a_stator = aerodynamic_losses.trailing_edge_losses_stator.K_TET(M_1, -10, alpha_2, h_vane_mean, stagger_angle_stator, r_meanline, pitch_axial_chord_ratio_stator)
                                            K_stator = K_p_stator + K_s_stator + K_TET_stator

                                            # CALCULATE LOSSES - Rotor
                                            K_p_rotor, pitch_chord_ratio_rotor, K_accel_rotor, stagger_angle_rotor, pitch_chord_ratio_rotor, pitch_axial_chord_ratio_rotor, K_1_rotor, K_2_rotor =  aerodynamic_losses.profile_losses.calc_K_p(M_2_rel, M_3_rel, M_2_rel_hub, P_2, P_3, r_tip, r_hub,  beta_2, beta_3, 0.80)
                                            K_s_rotor = aerodynamic_losses.secondary_losses.calc_K_s(K_accel_rotor, AR_rotor, beta_2, beta_3)
                                            K_TET_rotor, N_rotor, c_true_rotor, c_a_rotor = aerodynamic_losses.trailing_edge_losses_rotor.K_TET(M_2_rel, beta_2, beta_3, h, stagger_angle_rotor, r_meanline, pitch_axial_chord_ratio_rotor)
                                            K_rotor = K_p_rotor + K_s_rotor + K_TET_rotor

                                            eta_tt_new = aerodynamic_losses.efficiency_calculations(K_stator, K_rotor, M_2, M_3_rel, C_2, V_3)
                                            delta_n_new, eta_final_new = aerodynamic_losses.efficiency_final(eta_tt_new, h, beta_3, r_tip, r_meanline)

                                            # OFF DESIGN LOSSES
                                            eta_tt_od_new, delta_n_od_new, eta_final_od_new = aerodynamic_losses.losses_off_design(K_p_rotor, K_s_rotor, K_stator, K_1_rotor, K_2_rotor, pitch_chord_ratio_rotor, pitch_axial_chord_ratio_rotor, c_true_rotor, alpha_2, beta_2, beta_3, incidence_2,  M_2_rel_od, M_3_rel_od, h, stagger_angle_rotor, r_meanline, r_tip, C_2, V_3)
                                            
                                            # OPTIMIZATION FUNCTION
                                            function_value = (1/eta_final_new) + (eta_final_new - eta_final_od_new)

                                            if eta_final_od_new > 0:
                                                data_2.append((T_3_od,rho_3_od, P_3_od, alpha_3_od, flow_coeff_2_od, incidence_2, v_2_od, alpha_3_od,C_w_3_od,C_a_3_od,U_mean_od,flow_coeff_3_od, work_od_cw, work_od_vw, M_2_rel_od, M_3_rel_od,
                                                            eta_final, eta_final_new, eta_final_od, eta_final_od_new, function_value))

    df = pd.DataFrame(data, columns = ['stage_loading','reaction_mean','work_part_a','work_check_cw','work_check_vw','T_1', 'P_1', 'rho_1', 'A_1','C_1', 'C_a_1','M_3','T_3','P_3', 'rho_3', 'C_3', 'alpha_3','U',
                                    'C_a_3', 'C_w_3',  'V_3', 'V_w_3', 'flow_coefficient_3', 'beta_3', 'a_3', 'M_3_rel', 'A_3',
                                    'T_2', 'P_2','P_02','P_02_rel' ,'P_03_rel','rho_2', 'A_2', 'C_a_2', 'flow_coefficient_2', 'a_2', 'V_w_2', 'beta_2', 'V_2', 'C_w_2', 'C_2', 'alpha_2', 'M_2', 'M_2_rel',
                                    'N', 'omega', 'r_hub', 'r_tip', 'r_meanline', 'h',
                                    'alpha_2_hub', 'alpha_3_hub', 'beta_2_hub', 'beta_3_hub', 'U_hub', 'V_3_hub', 'C_2_hub', 'M_2_rel_hub', 'M_2_hub', 'reaction_hub',
                                    'alpha_2_tip', 'alpha_3_tip', 'beta_2_tip', 'beta_3_tip', 'U_tip', 'V_3_tip', 'C_2_tip', 'M_2_rel_tip', 'M_2_tip',
                                    'K_p_stator', 'K_s_stator', 'K_TET_stator', 'K_stator', 'N_stator',
                                    'K_p_rotor', 'K_s_rotor', 'K_TET_rotor', 'K_rotor', 'N_rotor', 'K_1_rotor', 'K_2_rotor', 'pitch_chord_ratio_rotor', 'c_true_rotor', 'c_a_rotor','stagger_angle_rotor','pitch_axial_chord_ratio_rotor',
                                    'eta_tt', 'delta_n', 'eta_final',
                                    'eta_tt_od', 'delta_n_od', 'eta_final_od',
                                    'work_od','rho_3_od', 'T_3_od', 'P_3_od', 'alpha_3_od', 'flow_coeff_2_od', 'incidence_2', 'v_2_od', 'alpha_3_rel_od', 'C_w_3_od', 'C_a_3_od', 'U_mean_od', 'flow_coeff_3_od', 'M_2_rel_od', 'M_3_rel_od'
                                    ])

    df_2 = pd.DataFrame(data_2, columns = ['T_3_od','rho_3_od', 'P_3_od', 'alpha_3_od', 'flow_coeff_2_od', 'incidence_2', 'v_2_od', 'alpha_3_od','C_w_3_od','C_a_3_od','U_mean_od','flow_coeff_3_od', 'work_od_cw', 'work_od_vw', 'M_2_rel_od', 'M_3_rel_od',
                                                    'eta_final', 'eta_final_new', 'eta_final_od', 'eta_final_od_new', 'function_value'])
    

    return df, df_2


run int main

In [4]:
df, df_optimize = int_main(range_degree_reaction, range_stage_loading, range_alpha_3, range_AN_squared, range_zweifel_vane, range_zweifel_rotor)

Splitting the main dataframe

In [5]:
df_losses_meanline = df[['K_p_stator', 'K_s_stator', 'K_TET_stator', 'K_stator', 'N_stator','K_p_rotor', 'K_s_rotor', 'K_TET_rotor', 'K_rotor', 'N_rotor', 'eta_tt', 'delta_n', 'eta_final',
                         'eta_tt_od', 'delta_n_od', 'eta_final_od']]
df_off_design = df[['work_od','rho_3_od', 'T_3_od', 'P_3_od', 'alpha_3_od', 'flow_coeff_2_od', 'incidence_2', 'v_2_od', 'alpha_3_rel_od', 'C_w_3_od', 'C_a_3_od', 'U_mean_od', 'flow_coeff_3_od'
                    , 'M_2_rel_od', 'M_3_rel_od']]

Write results to file

In [6]:
df.to_csv('_outputs/output_global.csv', index=False)
df_losses_meanline.to_csv('_outputs/output_losses_meanline.csv', index=False)
df_off_design.to_csv('_outputs/output_off_design.csv', index=False)
df_optimize.to_csv('_outputs/output_optimize.csv', index=False)